# Calcium imaging data from interneurons

In this notebook we will work with the data from an example animal in which interneurons were recorded using a calcium sensor (GCaMP) during a fear conditioning task. 

The mouse was presented with 5 tones (CS+) followed immediately by 5 mild aversive stimuli (US). 

We will look at the different cells responses to these two stimuli.

### Make sure to select the kernel

## We always first load our packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os #Added 
import pynapple as nap #this is pip repository
import scipy
import pickle #pip
import seaborn as sns

## Where are we in terms of files
Is it the same as where you have the data file? 
Make sure you are situated where the data is, so that you can read it

In [ ]:
os.getcwd()

#Uncomment the followning line if you need to change to a different path 
#os.chdir(path)

## Lets load the data

In [ ]:
# Load data
with open('InterneuronData.pkl', 'rb') as f:
    IntData = pickle.load(f)

#Timestamps of each frame recorded
scan_time = IntData['Timestamps']

#Fluorescence values of each cell
traces = IntData['Traces']

#Start time of the tones
CSonsets = IntData['CSplus']

#Start time of the unconditioned stimuli (US)
USonsets = IntData['US']

If you only have numpy (optional)

In [ ]:
#Timestamps of each frame recorded
#scan_time = np.load('Interneuron_timestamps.npy')

#Fluorescence values of each cell
#traces = np.load('Interneuron_traces.npy')

#Start time of the tones
#CSonsets = np.load('Interneuron_CSplus_onset.npy')


#Start time of the unconditioned stimuli (US)
#USonsets = np.load('Interneuron_US_onset.npy')


## What is inside?

In [ ]:
IntData

## How many tones and unconditioned stimuli were applied?

In [ ]:
print(f"{len(CSonsets)} tones were presented")
print(f" followed by {len(USonsets)} unconditioned stimuli")

## How long was the session? what was the frame rate? 20, 30 or 40 Hz?

In [ ]:
len(scan_time)

In [ ]:
scan_time[-1]/60 #The session lasted  20 min

In [ ]:
scan_time[1]-scan_time[0] # every frame is 50 ms apart, what is then the frame rate?

## How many cells did we record? Does the length of the traces matches the length of the timestamps?

In [ ]:
traces.shape

## Lets preprocess the traces

In [ ]:

#Zscore the traces
ca_z=nap.TsdFrame(t=scan_time,d=scipy.stats.zscore(traces,axis=0)) 
print(ca_z.shape)
ca_z

## Let's plot some traces!

In [ ]:
#Plot
fig, ax = plt.subplots(1,1, sharex=True,figsize=(17,6))

#Select a few cells
temp_cell=[2,6,11,12,13,22,27,42]

#Plot them!
plt.plot(scan_time,ca_z[:,temp_cell]+np.arange(0,len(temp_cell))*9,color="k",linewidth=.8)
plt.xlabel('Timesteps', fontsize=14)



# Now lets look at the peristimulus activity, what was the activity of the cells when the tone and unconditioned stimulus were happening?

## 1. Preprocessing 
### We can also z-score the traces by hand to understand what we are doing

In [ ]:
#Transpose matrix
traces = traces.T
print(traces.shape)
# CAREFUL! Run this only once, or you will transpose the matrix again and the dimensions would be wrong
#Shape should  be (44, 23995)

### To z score we get the mean and standard deviation of each neurons activity 
We normalize each cell to its own mean and standard deviation

In [ ]:
#We get one mean and std per cell
means = np.mean(traces, 1).reshape(traces.shape[0], 1)
stdevs = np.std(traces, 1).reshape(traces.shape[0], 1)

print(f"We have {len(means)} means")
print(f"We have {len(stdevs)} stds, one per each cell")

# Zscore
traces = (traces - means) / stdevs

## 2. Lets now find the closest timestamp to our stimuli onset

In [ ]:
closest_values = np.array([np.abs(scan_time - x).argmin() for x in CSonsets])
 #These are indexes of the closest timepoint in the timestamps to the CS plus onsets

## 3. Now we need to extract the traces around our stimuli and baseline the activity to the period before

In [ ]:
input_data = []

baseline = 30 #Time in seconds
SignalPeriod = 45 #Time in seconds
FrameRate = 20 # How many frames are we acquiring per second? 

for i in closest_values:
    #Divide the traces in baseline (pre event) and signal (post event)      
    pre_event_slice = traces[:, i - baseline*FrameRate:i]
    post_event_slice = traces[:, i-1:i + SignalPeriod*FrameRate]
            
    # baseline the slices (remove the pre-event means)
    pre_event_means = np.mean(pre_event_slice, 1).reshape(pre_event_slice.shape[0], 1)
    post_event_slice -= pre_event_means
    pre_event_slice -= pre_event_means
            
    # Concatenate pre-event and post-event together into a single trace
    concatenated_slice = np.concatenate([pre_event_slice, post_event_slice], axis=1) 
    input_data.append(concatenated_slice)

#Shape before
print(len(input_data)) # This is a list of 5 elements, each element of 44 neurons x 1501 timestamps

#We need to change it into a 3d matrix  (nStimuli, nCells, nTimestamps)
input_data=np.array(input_data)[:,:,:]

print(f'the shape of the matrix is {input_data.shape}, which accounts for 5 stimuli onset, 44 cells recorded, and 1501 timepoints')


## Select a few cells and average their peristimulus activity
### Are all the cells doing the same? are they all activated or inhibited by the CS+ and US presentation?

In [ ]:
#Mean activity of each cell to all stimuli presentation
cellMeans = np.mean(input_data,0)
print(cellMeans.shape)
print(f'the matrix cellMeans has 44 rows, one for each cell and 1501 columns, which are all the fluorescence values')

#Time of the peristumulus events
t=np.arange(-baseline,SignalPeriod+1/FrameRate, 1/FrameRate)

#Select a few cells
temp_cell=[6,12,22,27,42]

#Create the subplots
fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Loop through the cells
for i, cell in enumerate(temp_cell):

    #Get the data of each cell
    cell_data = cellMeans[cell,:]
    
    # Plot the mean 
    axes[i].plot(t,cell_data, color = 'black')
    
    # Set the title for each subplot
    axes[i].set_title(f"Cell: {cell+1}")
    axes[i].set_ylim(-3.5, 9)
    axes[i].plot(0, -3, 'b^',ms= 15)
    axes[i].plot(30, -3, 'r^',ms = 15)  
    

# Adjust the space between subplots
plt.subplots_adjust(hspace=0.1)  # Increase hspace for more space between subplots

# Add a single x-label for the entire figure
fig.text(0.5, 0.04, 'Time', ha='center', va='center', fontsize=14)

# Add a single y-label for the entire figure
fig.text(0.04, 0.5, 'Z- score', ha='center', va='center', rotation='vertical', fontsize=14)
fig.suptitle('Mean response to all CS-US presentations per cell')
# Display the plots
plt.show()


## Lets plot now the average of all cells to each stimulus presentation
### Does the average looks the same as the individual responses? what do you think? 

In [ ]:
#Mean activity of ALL cells to each stimuli presentation
stimMeans = np.mean(input_data,1)
print(stimMeans.shape)
print(f'This time the matrix stimMeans has 5 rows, one for each stimulus and 1501 columns, which are all the mean fluorescence values')
nCS = stimMeans.shape[0]

#Time of the peristumulus events
t=np.arange(-baseline,SignalPeriod+1/FrameRate, 1/FrameRate)

#Create the subplots
fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Loop through the stimuli
for i in range(nCS):

    #Get the data of each stimulus 
    stim_data = stimMeans[i,:]
    
    # Plot the mean 
    axes[i].plot(t,stim_data, color = 'black')
    
    # Set the title for each subplot
    axes[i].set_title(f"CS+ US pairing {i+1}")
    axes[i].set_ylim(-1, 2)
    axes[i].plot(0, -0.75, 'b^',ms= 15)
    axes[i].plot(30, -0.75, 'r^',ms = 15)    

# Add a single x-label for the entire figure
fig.text(0.5, 0.04, 'Time', ha='center', va='center', fontsize=14)

# Add a single y-label for the entire figure
fig.text(0.04, 0.5, 'Z- score', ha='center', va='center', rotation='vertical', fontsize=14)
fig.suptitle('Mean response of ALL cells to each CS+ US pairing')
# Display the plots
plt.show()

# Finally, we can also look at the US independently of the CS+ activity!
## what does the average activity to the US looks like? do you see any trend?

## 1. Collect peristimulus data around US only

In [ ]:
#Find closest values to the US onset
closest_values = np.array([np.abs(scan_time - x).argmin() for x in USonsets])

US_data = []

baseline = 5 #Time in seconds
SignalPeriod = 15 #Time in seconds
FrameRate = 20 # How many frames are we acquiring per second? 

for i in closest_values:
            
    pre_event_slice = traces[:, i - baseline*FrameRate:i]
    post_event_slice = traces[:, i-1:i + SignalPeriod*FrameRate]
            
    # Normalize the slices (remove the pre-event means)
    pre_event_means = np.mean(pre_event_slice, 1).reshape(pre_event_slice.shape[0], 1)
    post_event_slice -= pre_event_means
    pre_event_slice -= pre_event_means
            
    # Concatenate pre-event and post-event slices
    concatenated_slice = np.concatenate([pre_event_slice, post_event_slice], axis=1) 
    US_data.append(concatenated_slice)

#Shape before
print(len(US_data))

#Turning it into a 3d matrix 
US_data=np.array(US_data)[:,:,:]

print(f'the shape of the matrix is {US_data.shape}, which accounts for 5 stimuli onset, 44 cells recorded, and 1501 timepoints')

## 2. Plot the average of all cells in this mouse

In [ ]:
#Mean activity of ALL cells to each stimuli presentation
stimMeans = np.mean(US_data,1)
print(stimMeans.shape)
print(f'This time the matrix stimMeans has 5 rows, one for each stimulus and 1501 columns, which are all the mean fluorescence values')

#Time of the peristumulus events
t=np.arange(-baseline,SignalPeriod+1/FrameRate, 1/FrameRate)

#Create the subplots
fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Loop through the cells
for i in range(nCS):

    #Get the data of each stimulus
    stim_data = stimMeans[i,:]
    
    # Plot the mean 
    axes[i].plot(t,stim_data, color = 'black')
    
    # Set the title for each subplot
    axes[i].set_title(f"US presentation {i+1}")
    axes[i].set_ylim(-1, 2)
    axes[i].plot(0, -0.75, 'r^',ms = 15)    

# Adjust the space between subplots
plt.subplots_adjust(hspace=0.5)  # Increase hspace for more space between subplots

# Add a single x-label for the entire figure
fig.text(0.5, 0.04, 'Time', ha='center', va='center', fontsize=14)

# Add a single y-label for the entire figure
fig.text(0.04, 0.5, 'Z- score', ha='center', va='center', rotation='vertical', fontsize=14)
fig.suptitle('Mean response of ALL cells to each US presentations ')
# Display the plots
plt.show()

# Letss se all cells in a heatmap!

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Create custom colormap
cmapcustom = LinearSegmentedColormap.from_list(
    "custom_diverge",
    ["blue", "white", "red"],
    N = 64
)

## Lets sort the cells according to their response to the first US presentation.
### The cells that were activated to the first US are also activated afterwards? what do you think?
### Try aligning the activity to another US presentation by changing the UStoAlign value

In [ ]:
#Order the cell according to the response from highest to lowest to the first US presentation 

#To which US do you want to sort the cells?
UStoAlign = 0

#Obtain mean response to the first US presentation  and sort the cells from max response to min response
meansStim1 = np.mean(US_data[UStoAlign,:,5*FrameRate+1:10*FrameRate],1)
ascending_indices = meansStim1.argsort() 
descending_indices = ascending_indices[::-1] 


selected_labels = ["-5", "0", "15"]

#Plot heatmaps looking at all cells reponses across all 5 US presentations
f, axes = plt.subplots(1, 5, figsize=(15, 5))

for stim in range(nCS):
    if stim == 4:
        sns.heatmap(US_data[stim,descending_indices,:], cmap=cmapcustom,center=0, vmax=6, vmin= -4,ax=axes[stim],xticklabels=False, yticklabels=False )  # defaults include a colorbar
        axes[stim].set_title(f"US presentation {stim+1}")
        axes[stim].set_xticks([0,101,401])
        axes[stim].set_xticklabels(selected_labels)
    if stim == 0: 
        sns.heatmap(US_data[stim,descending_indices,:], cmap=cmapcustom,center=0, vmax=6, vmin= -4,ax=axes[stim], cbar = False,xticklabels=False)
        axes[stim].set_title(f"US presentation {stim+1}")
        axes[stim].set_xticks([0,101,401])
        axes[stim].set_xticklabels(selected_labels)
        axes[stim].set_xlabel("Time")
        axes[stim].set_ylabel("Cells")

        continue

    sns.heatmap(US_data[stim,descending_indices,:], cmap=cmapcustom,center=0, vmax=6, vmin= -4,ax=axes[stim], cbar = False,xticklabels=False, yticklabels=False )
    axes[stim].set_title(f"US presentation {stim+1}")
    axes[stim].set_xticks([0,101,401])
    axes[stim].set_xticklabels(selected_labels)
    pos = axes[stim].get_position()
